In [18]:
%matplotlib inline
import pandas as pd
import numpy as np
import json

In [31]:
wyniki = pd.read_csv("../dane/przetworzone/sumy_laureaty_2014.csv", low_memory=False)

In [32]:
wyniki.head()

,Unnamed: 0,id_szkoly,plec,rocznik,dysleksja,biologia_laureat,biologia_podstawowa,biologia_rozszerzona,chemia_laureat,chemia_podstawowa,...,j_polski_rozszerzona,matematyka_laureat,matematyka_rozszerzona,WOS_laureat,WOS_rozszerzona,informatyka_podstawowa,j_angielski_podstawowa,j_polski_podstawowa,matematyka_podstawowa,WOS_podstawowa
0,2353129,24097,k,1995,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25,4,NaN
1,17283699,72837,k,NaN,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,6,NaN
2,2347442,24111,k,1995,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11,15,3,NaN
3,1939422,21795,k,1994,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,4,NaN
4,2359723,25664,k,1995,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,15,39,13,NaN


In [30]:
szkoly = pd.read_csv("../dane/szkoly2014.csv")

In [33]:
szkoly.head()

,id_szkoly,typ_szkoly,publiczna,dla_doroslych,specjalna,przyszpitalna,artystyczna,rok,id_szkoly_oke,nazwa_szkoly,...,miejscowosc,pna,poczta,wielkosc_miejscowosci,matura_miedzynarodowa,teryt_szkoly,wojewodztwo_szkoly,powiat_szkoly,gmina_szkoly,rodzaj_gminy
0,25302,LO,False,True,False,False,NaN,2014,146502-0810Z,Liceum Ogólnokształcące dla Dorosłych nr 48 MGM,...,Warszawa,01-494,Warszawa,1724404,False,146502,mazowieckie,Warszawa,Bemowo,dzielnica m.st. Warszawy
1,25300,LO,False,False,False,False,NaN,2014,146502-0610L,Sportowe Liceum Ogólnokształcące nr 67 MGM,...,Warszawa,01-494,Warszawa,1724404,False,146502,mazowieckie,Warszawa,Bemowo,dzielnica m.st. Warszawy
2,25298,LO,True,False,False,False,NaN,2014,146502-01104,LXXVIII Liceum Ogólnokształcące im. Marii Pawl...,...,Warszawa,01-391,Warszawa,1724404,False,146502,mazowieckie,Warszawa,Bemowo,dzielnica m.st. Warszawy
3,25499,LO,False,False,False,False,NaN,2014,146502-04107,Liceum Ogólnokształcące Niepubliczne nr 29 im....,...,Warszawa,01-728,Warszawa,1724404,False,146502,mazowieckie,Warszawa,Bemowo,dzielnica m.st. Warszawy
4,43454,SP,False,False,NaN,NaN,NaN,2014,146502-04P0J,Szkoła Podstwawowa nr 24,...,Warszawa,01-355,Warszawa,1724404,NaN,146502,mazowieckie,Warszawa,Bemowo,dzielnica m.st. Warszawy


In [34]:
df = wyniki.merge(szkoly, how='left', on="id_szkoly")

In [35]:
df['wiek'] = (2014 - df.rocznik)

In [36]:
def wiek_uprosc(wiek):
    if wiek <= 18:
        return "18 lub mniej"
    elif wiek == 19:
        return "19"
    elif wiek == 20:
        return "20"
    else:
        return "21 i więcej"
    
df['wiek_prosciej'] = df['wiek'].apply(wiek_uprosc)

In [37]:
for k, v in df.groupby(["plec", "dysleksja"]).size().iterkv():
    print(k, v)

('k', False) 156873
('k', True) 7756
('m', False) 119783
('m', True) 13450


In [25]:
def get_coincidence_graph(df):
    
    total = len(df)
    
    nodes = []
    node_ref = {}
    for col in df.columns:
        for name, count in df[col].value_counts().iterkv():
            node_ref[(name, col)] = len(nodes)
            nodes.append({"name": str(name), "category": col, "count": int(count)})
            
    edges = []
    for col1 in df.columns:
        for col2 in df.columns:
            if col1 < col2:
                for (name1, name2), count in df.groupby([col1, col2]).size().iterkv():
                    i1 = node_ref[(name1, col1)]
                    i2 = node_ref[(name2, col2)]
                    edges.append({"source": i1,
                                  "target": i2,
                                  "count": int(count),
                                  "oe": (float(count) * total) / (nodes[i1]["count"] * nodes[i2]["count"]) })             
                
    return {"nodes": nodes, "links": edges}

In [38]:
get_coincidence_graph(df[["plec", "wiek_prosciej", ""]])

{'links': [{'count': 1618, 'oe': 1.2773202197562152, 'source': 0, 'target': 5},
  {'count': 109625, 'oe': 1.1577831323565368, 'source': 0, 'target': 2},
  {'count': 44011, 'oe': 0.7986302410911573, 'source': 0, 'target': 3},
  {'count': 11514, 'oe': 0.7330061723588208, 'source': 0, 'target': 4},
  {'count': 694, 'oe': 0.6769594392129653, 'source': 1, 'target': 5},
  {'count': 63194, 'oe': 0.8246608276270304, 'source': 1, 'target': 2},
  {'count': 56572, 'oe': 1.2684342359204395, 'source': 1, 'target': 3},
  {'count': 14508, 'oe': 1.1412239084505855, 'source': 1, 'target': 4}],
 'nodes': [{'category': 'plec', 'count': 166768, 'name': 'k'},
  {'category': 'plec', 'count': 134968, 'name': 'm'},
  {'category': 'wiek_prosciej', 'count': 172819, 'name': '19'},
  {'category': 'wiek_prosciej', 'count': 100583, 'name': '20'},
  {'category': 'wiek_prosciej', 'count': 28670, 'name': '21 i więcej'},
  {'category': 'wiek_prosciej', 'count': 2312, 'name': '18 lub mniej'}]}

In [23]:
df.groupby(["plec", "rocznik"]).size().order(ascending=False).head()

plec  rocznik
k     1995       109625
m     1995        63194
      1994        56572
k     1994        44011
m     1993         6354
dtype: int64

In [39]:
zmienne = ["plec", "wiek_prosciej", "dysleksja", "typ_szkoly", "rodzaj_gminy", "publiczna"]
graf = get_coincidence_graph(df[zmienne])
json.dump(graf, open("../dane/przetworzone/koincydencje_demograficzne.json", 'w'), indent=2)
json.dump(graf, open("../d3viz/data/koincydencje_demograficzne.json", 'w'), indent=2)